In [1]:
def medrank(lists):
    n_lists = len(lists)
    list_len = min(map(lambda x: len(x), lists))
    rank = dict()
    for idx in range(list_len):
        for l_idx in range(n_lists):
            item = lists[l_idx][idx]
            if item in rank:
                rank[item][l_idx] = idx+1
            else:
                x = [np.inf for _ in range(n_lists)]
                x[l_idx] = idx+1
                rank[item] = x
    filtered_keys = list(filter(lambda k: rank[k].count(np.inf) < n_lists / 2, rank))
    rank = {k: rank[k] for k in filtered_keys}
    for item in rank:
        rank[item] = np.median(rank[item])
    final_rank = sorted(rank.keys(), key=lambda z: rank[z])
    if len(final_rank) < 10:
        for idx in range(list_len):
            for l in lists:
                if l[idx] not in final_rank:
                    final_rank.append(l[idx])
                    if len(final_rank) >= 10:
                        return final_rank
    return final_rank

In [2]:
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np
import pandas as pd
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from RecSys2022.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from RecSys2022.Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
import RecSys2022.Utils.DataReader as dr

URM_all = dr.load_urm("../data/URMtries/RatingsBinary.csv")
#ICM_all = dr.load_icm("../data/data_ICM_type_truncated.csv")
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 930 ( 2.2%) Users that have less than 1 test interactions


In [3]:
EaseRecommender = EASE_R_Recommender(URM_train=URM_all, sparse_threshold_quota=1.0)
EaseRecommender.fit(topK=478, l2_norm=119.83582089765935)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 5.76 min
EASE_R_Recommender: Detected model matrix to be sparse, changing format.


In [4]:
SlimenRecommender = SLIMElasticNetRecommender(URM_train=URM_all)
SlimenRecommender.fit(topK=500, l1_ratio=0.001, alpha=0.01)

SLIMElasticNetRecommender: Processed 3449 (14.1%) in 5.00 min. Items per second: 11.49
SLIMElasticNetRecommender: Processed 7170 (29.3%) in 10.00 min. Items per second: 11.94
SLIMElasticNetRecommender: Processed 11252 (45.9%) in 15.00 min. Items per second: 12.50
SLIMElasticNetRecommender: Processed 15295 (62.4%) in 20.00 min. Items per second: 12.74
SLIMElasticNetRecommender: Processed 19380 (79.1%) in 25.00 min. Items per second: 12.92
SLIMElasticNetRecommender: Processed 23264 (94.9%) in 30.00 min. Items per second: 12.92
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 31.54 min. Items per second: 12.95


In [5]:
recommenders = [EaseRecommender, SlimenRecommender]

In [14]:
class HybridRecommender:
    def __init__(self, recommenders, urm_train):
        self.recommenders = recommenders
        self.urm_train = urm_train

    def recommend_lists_medrank(self, user_id, at=10, exclude_seen=True):
        recommendations = [recommender.recommend(user_id, cutoff=at, remove_seen_flag=True) for recommender in self.recommenders]
        return medrank(recommendations)[:at]

In [15]:
hybrid = HybridRecommender(recommenders=recommenders, urm_train=URM_all)

In [16]:
test_users = pd.read_csv('../RecSys2022/data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
41111,41624
41112,41625
41113,41626
41114,41627


In [17]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(hybrid.recommend_lists_medrank(user))

In [18]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission-medrank.csv', index=False)